In [132]:
# Data Processing Librearies 
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import xlrd
import time
import pandas as pd
import operator
import glob, os, shutil
from numpy import math
import math
from datetime import datetime, timedelta, date

#Server EMail
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os.path


from tkinter import filedialog
from tkinter import *
root = Tk()


In [135]:
# Get today´s date in format: dd/mm/YY
today = date.today()
TodayDate = today.strftime("%d_%m_%Y")



# File directory
root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes=[("Excel files", "*.xlsx; *.xls")])
FileDirectory = root.filename

# Base_line: read
BaseLine = pd.read_excel(FileDirectory,index_col=0).reset_index()

#Renombrar columnas de archivo virgen para ser proecesado en horas
BaseLine.rename(columns={"Unidad Energía":"FECHA",'KWh':'H0','Unnamed: 2':'H1','Unnamed: 3':'H2','Unnamed: 4':'H3','Unnamed: 5':'H4','Unnamed: 6':'H5','Unnamed: 7':'H6','Unnamed: 8':'H7','Unnamed: 9':'H8','Unnamed: 10':'H9','Unnamed: 11':'H10','Unnamed: 12':'H11','Unnamed: 13':'H12','Unnamed: 14':'H13','Unnamed: 15':'H14','Unnamed: 16':'H15','Unnamed: 17':'H16','Unnamed: 18':'H17','Unnamed: 19':'H18','Unnamed: 20':'H19','Unnamed: 21':'H20','Unnamed: 22':'H21','Unnamed: 23':'H22','Unnamed: 24':'H23','Unnamed: 25':'TOTAL'}, inplace=True)

#Base line: Columns selection  
BaseLine = BaseLine[['FECHA','H0','H1','H2','H3','H4','H5','H6','H7','H8','H9','H10','H11','H12','H13','H14','H15','H16','H17','H18','H19','H20','H21','H22','H23']]
#Base line: Last 105 days of measurements
BaseLine = BaseLine.iloc[len(BaseLine['FECHA'])-61:len(BaseLine['FECHA'])-1].reset_index()
#Reemplazar los valores NaN porceros.
BaseLine = BaseLine.fillna(0)

#Load Holidays
# File directory
FileDirectory = '../Colombian_Holidays/Holidays.xlsx'

# Holidays: read
HolidayList = pd.read_excel(FileDirectory,index_col=0).reset_index()

#________________________________________________
#1. Preparación de datos

#1.1 Sumar consumo diario (Ct)
#Base line: Daily consuption
BaseLine['TOTAL'] = round(BaseLine['H0'] + BaseLine['H1'] + BaseLine['H2'] + BaseLine['H3'] + BaseLine['H4'] + BaseLine['H5'] + BaseLine['H6'] + BaseLine['H7'] + BaseLine['H8'] + BaseLine['H9'] + BaseLine['H10'] + BaseLine['H11'] + BaseLine['H12'] + BaseLine['H13'] + BaseLine['H14'] + BaseLine['H15'] + BaseLine['H16'] + BaseLine['H17'] + BaseLine['H18'] + BaseLine['H19'] + BaseLine['H20'] + BaseLine['H21'] + BaseLine['H22'] + BaseLine['H23'],2) 

#1.2 Identificar qué día de la semana es cada fecha y procesa los festivos
# Base line: Days convertion/correction
BaseLine['WEEKDAY'] = pd.Series([0 for x in range(len(BaseLine['FECHA']))])
BaseLine['FESTIVOS'] = pd.Series([0 for x in range(len(BaseLine['FECHA']))])

#Asigna a cada fecha el día de la semana correspondiente
for MeasuredDays in range(len(BaseLine['FECHA'])):
    BaseLine['WEEKDAY'].iloc[MeasuredDays] = datetime.strptime(BaseLine['FECHA'].iloc[MeasuredDays], '%Y-%m-%d').weekday() + 1

    
#Identificar si el día es festivo (no domingo)
for MeasuredDays in range(len(BaseLine['FECHA'])):
    if BaseLine['FECHA'][MeasuredDays] in list(HolidayList['Holidays']):
        BaseLine['FESTIVOS'][MeasuredDays] = 1
        BaseLine['WEEKDAY'][MeasuredDays] = 8

#Reducción de datos para mayor velocidad
BaseLine_summary    = BaseLine[['FECHA','WEEKDAY','TOTAL']]
        
#2.1 Corrección de datos
#DEPURACIÓN INICIAL

#2.1.1 Dividir el data set por día de la semana
BaseLine_Mondays     = BaseLine_summary.loc[BaseLine_summary['WEEKDAY'] == 1]
BaseLine_Tuesdays    = BaseLine_summary.loc[BaseLine_summary['WEEKDAY'] == 2]
BaseLine_Wednesdays  = BaseLine_summary.loc[BaseLine_summary['WEEKDAY'] == 3]
BaseLine_Thursdays   = BaseLine_summary.loc[BaseLine_summary['WEEKDAY'] == 4]
BaseLine_Fridays     = BaseLine_summary.loc[BaseLine_summary['WEEKDAY'] == 5]
BaseLine_Saturdays   = BaseLine_summary.loc[BaseLine_summary['WEEKDAY'] == 6]
BaseLine_Sundays     = BaseLine_summary.loc[BaseLine_summary['WEEKDAY'] == 7]
BaseLine_Holydays    = BaseLine_summary.loc[BaseLine_summary['WEEKDAY'] == 8]

#2.1.2 Contar candidad de días del mismo tipo
Days_Monday    = len(BaseLine_Mondays)
Days_Tuesday   = len(BaseLine_Tuesdays) 
Days_Wednesday = len(BaseLine_Wednesdays)
Days_Thursday  = len(BaseLine_Thursdays)
Days_Friday    = len(BaseLine_Fridays)
Days_Saturday  = len(BaseLine_Saturdays)
Days_Sunday    = len(BaseLine_Sundays)
Days_Holyday   = len(BaseLine_Holydays)

#2.1.3 Contar porcentaje de días con consumo 0 MWh
Zero_Mondays    = len(BaseLine_Mondays.loc[BaseLine_Mondays['TOTAL'] == 0])/Days_Monday
Zero_Tuesdays   = len(BaseLine_Tuesdays.loc[BaseLine_Tuesdays['TOTAL'] == 0])/Days_Tuesday
Zero_Wednesdays = len(BaseLine_Wednesdays.loc[BaseLine_Wednesdays['TOTAL'] == 0])/Days_Wednesday
Zero_Thursdays  = len(BaseLine_Thursdays.loc[BaseLine_Thursdays['TOTAL'] == 0])/Days_Thursday
Zero_Fridays    = len(BaseLine_Fridays.loc[BaseLine_Fridays['TOTAL'] == 0])/Days_Friday
Zero_Saturdays  = len(BaseLine_Saturdays.loc[BaseLine_Saturdays['TOTAL'] == 0])/Days_Saturday
Zero_Sundays    = len(BaseLine_Sundays.loc[BaseLine_Sundays['TOTAL'] == 0])/Days_Sunday
Zero_Holydays   = len(BaseLine_Holydays.loc[BaseLine_Holydays['TOTAL'] == 0])/Days_Holyday

#2.1.4 Para días tipo con más de 20% de registros con cero la LBC es 0
if Zero_Mondays > 0.2:
    BaseLine_Mondays['TOTAL'] = 0
if Zero_Tuesdays > 0.2:
    BaseLine_Tuesdays['TOTAL'] = 0
if Zero_Wednesdays > 0.2:
    BaseLine_Wednesdays['TOTAL'] = 0
if Zero_Thursdays > 0.2:
    Days_Thursday['TOTAL'] = 0
if Zero_Fridays > 0.2:
    BaseLine_Fridays['TOTAL'] = 0
if Zero_Saturdays > 0.2:
    BaseLine_Saturdays['TOTAL'] = 0
if Zero_Sundays > 0.2:
    BaseLine_Sundays['TOTAL'] = 0
if Zero_Holydays > 0.2:
    BaseLine_Holydays['TOTAL'] = 0

#2.1.5 Completar día festivo (no domingo) con menos de 5 registro, por los valores menores del domingo.
if Zero_Holydays < 5:
    ValueMin1 = BaseLine_Sundays['TOTAL'].nsmallest(1).iloc[-1] #El menor valor del domingo
    ValueMin2 = BaseLine_Sundays['TOTAL'].nsmallest(2).iloc[-1] #El segundo menor valor del domingo
    DateMin1  = BaseLine_Sundays.loc[BaseLine_Sundays['TOTAL'] == BaseLine_Sundays['TOTAL'].nsmallest(1).iloc[-1]]['FECHA'].to_numpy()[0]
    DateMin2  = BaseLine_Sundays.loc[BaseLine_Sundays['TOTAL'] == BaseLine_Sundays['TOTAL'].nsmallest(2).iloc[-1]]['FECHA'].to_numpy()[0]
    min1 = [{'FECHA': DateMin1,'WEEKDAY':8,'TOTAL':ValueMin1}]
    min2 = [{'FECHA': DateMin2,'WEEKDAY':8,'TOTAL':ValueMin2}]
    BaseLine_Holydays = BaseLine_Holydays.append(min1,ignore_index=True,sort=False)
    BaseLine_Holydays = BaseLine_Holydays.append(min2,ignore_index=True,sort=False)

#TRANSFORMACIÓN DE VALORES IGUALES A CERO

    


C:\Users\USER\AppData\Local\Temp\ipykernel_9396\4227359768.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BaseLine['WEEKDAY'].iloc[MeasuredDays] = datetime.strptime(BaseLine['FECHA'].iloc[MeasuredDays], '%Y-%m-%d').weekday() + 1
C:\Users\USER\AppData\Local\Temp\ipykernel_9396\4227359768.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BaseLine['FESTIVOS'][MeasuredDays] = 1
C:\Users\USER\AppData\Local\Temp\ipykernel_9396\4227359768.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

date = BaseLine_Sundays.loc[BaseLine_Sundays['TOTAL'] == BaseLine_Sundays['TOTAL'].nsmallest(2).iloc[-1]]
a = date.get_value(48, 'FECHA', takeable=False) 
a

In [ ]:
#3  Cálculo del error de estimación de la LBC

#3.2 Sacar promedios simples por días
#Calcula la energía promedio  por día de la semana 
#En la regulación se conoce a cada Curve_Average como la "estimación de la LBC diaria".
SimpleAverage_summary   = BaseLine_summary[['TOTAL']].mean(axis=0)
SimpleAverage_Monday    = BaseLine_Mondays[['TOTAL']].mean(axis=0)
SimpleAverage_Tuesday   = BaseLine_Tuesdays[['TOTAL']].mean(axis=0)
SimpleAverage_Wednesday = BaseLine_Wednesdays[['TOTAL']].mean(axis=0)
SimpleAverage_Thursday  = BaseLine_Thursdays[['TOTAL']].mean(axis=0)
SimpleAverage_Friday    = BaseLine_Fridays[['TOTAL']].mean(axis=0)
SimpleAverage_Saturday  = BaseLine_Saturdays[['TOTAL']].mean(axis=0)
SimpleAverage_Sunday    = BaseLine_Sundays[['TOTAL']].mean(axis=0)

#3.3 Número de datos n del día tipo t: laboral, sábado, domingo o festivo,del total de la muestra m.
m           = len(BaseLine_summary['TOTAL'])
n_Monday    = len(BaseLine_Mondays['TOTAL'])
n_Tuesday   = len(BaseLine_Tuesdays['TOTAL'])
n_Wednesday = len(BaseLine_Wednesdays['TOTAL'])
n_Thursday  = len(BaseLine_Thursdays['TOTAL'])
n_Friday    = len(BaseLine_Fridays['TOTAL'])
n_Saturday  = len(BaseLine_Saturdays['TOTAL'])
n_Sunday    = len(BaseLine_Sundays['TOTAL'])

#3.4 Cáluclo intermedio para estimar un parámetro de la función RRMSE
Sumatory_Monday = 0
for n in range(n_Monday):
    Sumatory_Monday = Sumatory_Monday + (SimpleAverage_Monday-list(BaseLine_Mondays['TOTAL'])[n])**2

Sumatory_Tuesday = 0
for n in range(n_Tuesday):
    Sumatory_Tuesday = Sumatory_Tuesday + (SimpleAverage_Tuesday-list(BaseLine_Tuesdays['TOTAL'])[n])**2

Sumatory_Wednesday = 0
for n in range(n_Wednesday):
    Sumatory_Wednesday = Sumatory_Wednesday + (SimpleAverage_Wednesday-list(BaseLine_Wednesdays['TOTAL'])[n])**2

Sumatory_Thursday = 0
for n in range(n_Thursday):
    Sumatory_Thursday = Sumatory_Thursday + (SimpleAverage_Thursday-list(BaseLine_Thursdays['TOTAL'])[n])**2

Sumatory_Friday = 0
for n in range(n_Friday):
    Sumatory_Friday = Sumatory_Friday + (SimpleAverage_Friday-list(BaseLine_Fridays['TOTAL'])[n])**2

Sumatory_Saturday = 0
for n in range(n_Saturday):
    Sumatory_Saturday = Sumatory_Saturday + (SimpleAverage_Saturday-list(BaseLine_Saturdays['TOTAL'])[n])**2

Sumatory_Sunday = 0
for n in range(n_Sunday ):
    Sumatory_Sunday  = Sumatory_Sunday + (SimpleAverage_Sunday-list(BaseLine_Sundays['TOTAL'])[n])**2
    
#3.4 Cálculo del error de estimación de la LBC (se calculará como el valor relativo de la
#raíz cuadrada del error medio cuadrático de los datos, RRMSE (por sus siglas en inglés)

RRMSE_Monday       = math.sqrt((1/n_Monday)*(Sumatory_Monday))/((1/n_Monday)*sum(BaseLine_Mondays['TOTAL']))
RRMSE_Tuesday      = math.sqrt((1/n_Tuesday)*(Sumatory_Tuesday))/((1/n_Tuesday)*sum(BaseLine_Tuesdays['TOTAL']))
RRMSE_Wednesday    = math.sqrt((1/n_Wednesday)*(Sumatory_Wednesday))/((1/n_Wednesday)*sum(BaseLine_Wednesdays['TOTAL']))
RRMSE_Thursday     = math.sqrt((1/n_Thursday)*(Sumatory_Thursday))/((1/n_Thursday)*sum(BaseLine_Thursdays['TOTAL']))
RRMSE_Friday       = math.sqrt((1/n_Friday)*(Sumatory_Friday))/((1/n_Friday)*sum(BaseLine_Fridays['TOTAL']))
RRMSE_Saturday     = math.sqrt((1/n_Saturday)*(Sumatory_Saturday))/((1/n_Saturday)*sum(BaseLine_Saturdays['TOTAL']))
RRMSE_Sunday       = math.sqrt((1/n_Sunday)*(Sumatory_Sunday))/((1/n_Sunday)*sum(BaseLine_Sundays['TOTAL']))

#3.5 Cálculo de factor de corrección de la linea base para cada día

if RRMSE_Monday <= 0.05:
    MondayCorrectionFactor = 1
elif 0.05 < RRMSE_Monday <=0.2:
    MondayCorrectionFactor = (1-RRMSE_Monday)
elif 0.2 < RRMSE_Monday:
    MondayCorrectionFactor = 0
else:
    Print("Hay un error en el cálculo de la RRMSE del Lunes")
    
if RRMSE_Tuesday <= 0.05:
    TuesdayCorrectionFactor = 1
elif 0.05 < RRMSE_Tuesday <=0.2:
    TuesdayCorrectionFactor = (1-RRMSE_Tuesday)
elif 0.2 < RRMSE_Tuesday:
    TuesdayCorrectionFactor = 0
else:
    Print("Hay un error en el cálculo de la RRMSE del Martes")

if RRMSE_Wednesday <= 0.05:
    WednesdayCorrectionFactor = 1
elif 0.05 < RRMSE_Wednesday <=0.2:
    WednesdayCorrectionFactor = (1-RRMSE_Wednesday)
elif 0.2 < RRMSE_Wednesday:
    WednesdayCorrectionFactor = 0
else:
    Print("Hay un error en el cálculo de la RRMSE del Miércoles")

if RRMSE_Thursday <= 0.05:
    ThursdayCorrectionFactor = 1
elif 0.05 < RRMSE_Thursday <=0.2:
    ThursdayCorrectionFactor = (1-RRMSE_Thursday)
elif 0.2 < RRMSE_Thursday:
    ThursdayCorrectionFactor = 0
else:
    Print("Hay un error en el cálculo de la RRMSE del Jueves")
    
if RRMSE_Friday <= 0.05:
    FridayCorrectionFactor = 1
elif 0.05 < RRMSE_Friday <=0.2:
    FridayCorrectionFactor = (1-RRMSE_Friday)
elif 0.2 < RRMSE_Friday:
    FridayCorrectionFactor = 0
else:
    Print("Hay un error en el cálculo de la RRMSE del Viernes")

if RRMSE_Saturday <= 0.05:
    SaturdayCorrectionFactor = 1
elif 0.05 < RRMSE_Saturday <=0.2:
    SaturdayCorrectionFactor = (1-RRMSE_Saturday)
elif 0.2 < RRMSE_Saturday:
    SaturdayCorrectionFactor = 0
else:
    Print("Hay un error en el cálculo de la RRMSE del Sábado")

if RRMSE_Sunday <= 0.05:
    SundayCorrectionFactor = 1
elif 0.05 < RRMSE_Sunday <=0.2:
    SundayCorrectionFactor = (1-RRMSE_Sunday)
elif 0.2 < RRMSE_Sunday:
    SundayCorrectionFactor = 0
else:
    Print("Hay un error en el cálculo de la RRMSE del Domingo")
    
#3.5 Cálculo de la LBC diaria

LBC_Monday    = MondayCorrectionFactor*SimpleAverage_Monday
LBC_Tuesday   = TuesdayCorrectionFactor*SimpleAverage_Tuesday
LBC_Wednesday = WednesdayCorrectionFactor*SimpleAverage_Wednesday
LBC_Thursday  = ThursdayCorrectionFactor*SimpleAverage_Thursday
LBC_Friday    = FridayCorrectionFactor*SimpleAverage_Friday
LBC_Saturday  = SaturdayCorrectionFactor*SimpleAverage_Saturday
LBC_Sunday    = SundayCorrectionFactor*SimpleAverage_Sunday

#3.6 Estimación de la línea base de consumo de energía horaria
#No entendí
#XXXXXXXXXXXXXXXXXXXXXXX TRBAJAR AQUI XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

#4. VERIFICACIÓN DE DEMANDA DESCONECTADA
#4.1 verificación de la reducción de consumo de la demanda 
#desconectada por un usuario se realizará según la siguiente expresión:
#XXXXXXXXXXXXXXXXXXXXXXX TRBAJAR AQUI XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


#5 GUARDAR INFORMACIÓN CALCULADA DE LA FRONTERA EN ARCHIVO EXCEL


# File directory
FileDirectory = root.filename[:-5]+"AAAAA"+"_Results_RRMSE_version"+TodayDate+".xlsx"

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(FileDirectory, engine='xlsxwriter')


#1. Estimación de la LBC diaria
Estimacion_LBC_Diaria = pd.concat([SimpleAverage_Monday, 
                                   SimpleAverage_Tuesday,
                                   SimpleAverage_Wednesday,
                                   SimpleAverage_Thursday,
                                   SimpleAverage_Friday,
                                   SimpleAverage_Saturday,
                                   SimpleAverage_Sunday], 
                                   keys=['Lunes', 'Martes','Miércoles','Jueves','Vieres','Sábado','Domingo'],
                                   names =['Día','Estimacion_LBC_Diaria'])

Estimacion_LBC_Diaria.to_excel(writer, sheet_name='Estimacion_LBC_Diaria')

#2. Número de datos n del día tipo t
dias        = ['Lunes', 'Martes','Miércoles','Jueves','Vieres','Sábado','Domingo']
Num_dias    = [n_Monday,n_Tuesday, n_Wednesday,n_Thursday,n_Friday, n_Saturday,n_Sunday]
Numero_dias = pd.DataFrame()
Numero_dias['Día'] = dias
Numero_dias['Número de días'] = Num_dias
Numero_dias.set_index('Día')
Numero_dias.to_excel(writer, sheet_name='n_días')

#3. RRMSE diario
dias     = ['Lunes', 'Martes','Miércoles','Jueves','Vieres','Sábado','Domingo']
RRMSE    = [RRMSE_Monday,RRMSE_Tuesday, RRMSE_Wednesday,RRMSE_Thursday,RRMSE_Friday, RRMSE_Saturday,RRMSE_Sunday]
RRMSE_dias = pd.DataFrame()
RRMSE_dias['Día'] = dias
RRMSE_dias['RRMSE día'] = RRMSE
RRMSE_dias.set_index('Día')
RRMSE_dias.to_excel(writer, sheet_name='RRMSE_dias')

#4 CorrectionFactors
dias                 = ['Lunes', 'Martes','Miércoles','Jueves','Vieres','Sábado','Domingo']
CorrectionFactors    = [MondayCorrectionFactor,TuesdayCorrectionFactor, WednesdayCorrectionFactor,ThursdayCorrectionFactor,
                        FridayCorrectionFactor, SaturdayCorrectionFactor,SundayCorrectionFactor]
CorrectionFactor = pd.DataFrame()
CorrectionFactor['Día'] = dias
CorrectionFactor['Factores de corrección'] = CorrectionFactors
CorrectionFactor.set_index('Día')
CorrectionFactor.to_excel(writer, sheet_name='Factores de corrección')

#5 LBC diaria
dias    = ['Lunes', 'Martes','Miércoles','Jueves','Vieres','Sábado','Domingo']
LBCs    = [int(LBC_Monday),int(LBC_Tuesday),int(LBC_Wednesday),int(LBC_Thursday),int(LBC_Friday), int(LBC_Saturday), int(LBC_Sunday)]
LBC = pd.DataFrame()
LBC['Día'] = dias
LBC['Factores de corrección'] = LBCs
LBC.set_index('Día')
LBC.to_excel(writer, sheet_name='LBC diario')


# Close the Pandas Excel writer and output the Excel file.
writer.save()